<a href="https://colab.research.google.com/github/menaxyz/final_AI/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Shiyun Zhou CS481 final


Q1


In [21]:
import heapq


def find_path(came_from, start, goal):
    current = goal
    path = []

    while current != start:
        path.insert(0, current)
        current = came_from[current]

    path.insert(0, start)
    return path


def heuristic(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)


def find_neighbors(maze, current_position):
    x, y = current_position
    potential_neighbors = [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]#NSEW

    valid_neighbors = [
    (nx, ny) for nx, ny in potential_neighbors if
    0 <= nx < len(maze) and
    0 <= ny < len(maze[0]) and
    0 <= maze[nx][ny] < 2 ]

    return valid_neighbors


# A*
def a_star_search(maze, start, goals):
    frontier = []
    heapq.heappush(frontier, (0, start))
    before = {}
    cost = {}
    before[start] = None
    cost[start] = 0

    found_goals = []

    while frontier:

        _, current = heapq.heappop(frontier)
        if current in goals and current not in found_goals:
            found_goals.append(current)
            if len(found_goals) == len(goals):
                break

        #find neighbors
        for next in find_neighbors(maze, current):

            new_cost = cost[current] + 1

            if next not in cost or new_cost < cost[next]:
                cost[next] = new_cost
                priority = new_cost + heuristic(next, current)
                heapq.heappush(frontier, (priority, next))
                before[next] = current

    paths = [find_path(before, start, goal) if goal in before else [] for goal in goals]
    return paths


maze = [
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0],
    [0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0],
    [1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1],
    [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0],
    [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
    [0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
]

start = (10, 5)
goal1 = [(0, 5)]
goal2 = [(4, 2)]

path1 = a_star_search(maze, start, goal1)
print("Paths to goal1:", path1)

path2 = a_star_search(maze, start, goal2)
print("Paths to goal2:", path2)




Paths to goal1: [[(10, 5), (9, 5), (8, 5), (7, 5), (6, 5), (5, 5), (4, 5), (3, 5), (2, 5), (1, 5), (0, 5)]]
Paths to goal2: [[(10, 5), (9, 5), (8, 5), (7, 5), (6, 5), (5, 5), (4, 5), (4, 4), (4, 3), (4, 2)]]


Q2


In [25]:
import json

def generate_states():

    states = set()

    generate_recursive("X---X---O", states)

    states_list = list(states)

    states_dict = {state: get_child_states(state) for state in states_list}
    return states_dict


def all_possible_states(state, states):

    if state not in states:

        states.add(state)

        for i in range(len(state)):

            if state[i] == "-":

                for symbol in ['X', 'O']:
                    new_state = state[:i] + symbol + state[i+1:]
                    all_possible_states(new_state, states)


def get_child_states(state):

    child_states = []
    for i in range(len(state)):
        if state[i] == "-":
            for symbol in ['X', 'O']:
                new_state = state[:i] + symbol + state[i+1:]
                child_states.append(new_state)
    return child_states

def state_to_3x3_grid(state):
    return [list(state[i:i+3]) for i in range(0, len(state), 3)]

def save_to_json(states_dict, filename="tic_tac_toe_states.json"):
    formatted_states = {state: state_to_3x3_grid(state) for state in states_dict.keys()}
    with open(filename, 'w') as file:
        json.dump(formatted_states, file, indent=2)

def print_board(state):
    for row in state:
        print(row)

tic_tac_toe_states = generate_states()
save_to_json(tic_tac_toe_states)

print(" tic_tac_toe_states.json saved .")


 tic_tac_toe_states.json saved .


Q3

In [27]:

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.utils import to_categorical


(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

model = Sequential([
    Flatten(input_shape=(784,)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc * 100:.2f}%')


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
1500/1500 [==============================] - 11s 7ms/step - loss: 0.2886 - accuracy: 0.9181 - val_loss: 0.1553 - val_accuracy: 0.9554
Epoch 2/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.1288 - accuracy: 0.9617 - val_loss: 0.1148 - val_accuracy: 0.9652
Epoch 3/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0888 - accuracy: 0.9736 - val_loss: 0.1116 - val_accuracy: 0.9676
Epoch 4/10
1500/1500 [==============================] - 11s 7ms/step - loss: 0.0661 - accuracy: 0.9797 - val_loss: 0.0959 - val_accuracy: 0.9721
Epoch 5/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0504 - accuracy: 0.9841 - val_loss: 0.1021 - val_accuracy: 0.9713
Epoch 6/10
1500/1500 [==============================] - 8s 6ms/step - loss: 0.0395 - accuracy: 0.9879 - val_loss: 0.0867 - val_accuracy: 0.9746
Epoch 7/10
1500/1500 [==============================] - 7s 5ms/step -

Q4


In [34]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.datasets import fetch_california_housing


def load_data(test_split=0.2, seed=113):
    data = fetch_california_housing()
    x_train, x_test, y_train, y_test = train_test_split(data.data, data.target, test_size=test_split, random_state=seed)

    return (x_train, y_train), (x_test, y_test)

def preprocess_data(x_train, x_test):
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    return x_train_scaled, x_test_scaled

def build_regression_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test, epochs=50, batch_size=32):
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    y_pred = model.predict(x_test).flatten()

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    return mse, mae


def main():
    (x_train, y_train), (x_test, y_test) = load_data()

    x_train_scaled, x_test_scaled = preprocess_data(x_train, x_test)

    model = build_regression_model(input_shape=x_train_scaled.shape[1])
    mse, mae = train_and_evaluate_model(model, x_train_scaled, y_train, x_test_scaled, y_test)

    print(f'MSE: {mse:.2f}')
    print(f'MAE: {mae:.2f}')

if __name__ == "__main__":
    main()


129/129 [==============================] - 0s 2ms/step
MSE: 0.27
MAE: 0.35
